# Cookbook

In this notebook we'll take a look at a few common types of sequences to create.

## PromptTemplate + LLM

A PromptTemplate -> LLM is a core chain that is used in most other larger chains/systems.

In [19]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

In [2]:
model = ChatOpenAI()

In [3]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")

In [4]:
chain = prompt | model

In [5]:
chain.invoke({"foo": "bears"})

AIMessage(content='Why don\'t bears use cell phones? \n\nBecause they always get terrible "grizzly" reception!', additional_kwargs={}, example=False)

Often times we want to attach kwargs to the model that's passed in. Here's a few examples of that:

### Attaching Stop Sequences

In [6]:
chain = prompt | model.bind(stop=["\n"])

In [7]:
chain.invoke({"foo": "bears"})

AIMessage(content="Why don't bears use cell phones?", additional_kwargs={}, example=False)

### Attaching Function Call information

In [8]:
functions = [
    {
      "name": "joke",
      "description": "A joke",
      "parameters": {
        "type": "object",
        "properties": {
          "setup": {
            "type": "string",
            "description": "The setup for the joke"
          },
          "punchline": {
            "type": "string",
            "description": "The punchline for the joke"
          }
        },
        "required": ["setup", "punchline"]
      }
    }
  ]
chain = prompt | model.bind(function_call= {"name": "joke"}, functions= functions)

In [9]:
chain.invoke({"foo": "bears"}, config={})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'joke', 'arguments': '{\n  "setup": "Why don\'t bears wear shoes?",\n  "punchline": "Because they have bear feet!"\n}'}}, example=False)

## PromptTemplate + LLM + OutputParser

We can also add in an output parser to easily trasform the raw LLM/ChatModel output into a more workable format

In [11]:
from langchain.schema.output_parser import StrOutputParser

In [12]:
chain = prompt | model | StrOutputParser()

Notice that this now returns a string - a much more workable format for downstream tasks

In [13]:
chain.invoke({"foo": "bears"})

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

### Functions Output Parser

When you specify the function to return, you may just want to parse that directly

In [14]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
chain = (
    prompt 
    | model.bind(function_call= {"name": "joke"}, functions= functions) 
    | JsonOutputFunctionsParser()
)

In [15]:
chain.invoke({"foo": "bears"})

{'setup': "Why don't bears wear shoes?",
 'punchline': 'Because they have bear feet!'}

In [17]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
chain = (
    prompt 
    | model.bind(function_call= {"name": "joke"}, functions= functions) 
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [18]:
chain.invoke({"foo": "bears"})

"Why don't bears like fast food?"

## Passthroughs and itemgetter

Often times when constructing a chain you may want to pass along original input variables to future steps in the chain. How exactly you do this depends on what exactly the input is:

- If the original input was a string, then you likely just want to pass along the string. This can be done with `RunnablePassthrough`. For an example of this, see `LLMChain + Retriever`
- If the original input was a dictionary, then you likely want to pass along specific keys. This can be done with `itemgetter`. For an example of this see `Multiple LLM Chains`

In [19]:
from langchain.schema.runnable import RunnablePassthrough
from operator import itemgetter

## LLMChain + Retriever

Let's now look at adding in a retrieval step, which adds up to a "retrieval-augmented generation" chain

In [20]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough

In [21]:
# Create the retriever
vectorstore = Chroma.from_texts(["harrison worked at kensho"], embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [22]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [24]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

In [25]:
chain.invoke("where did harrison work?")

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'Harrison worked at Kensho.'

In [27]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = {
    "context": itemgetter("question") | retriever, 
    "question": itemgetter("question"), 
    "language": itemgetter("language")
} | prompt | model | StrOutputParser()

In [28]:
chain.invoke({"question": "where did harrison work", "language": "italian"})

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'Harrison ha lavorato a Kensho.'

## Multiple LLM Chains

This can also be used to string together multiple LLMChains

In [31]:
from operator import itemgetter

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template("what country is the city {city} in? respond in {language}")

chain1 = prompt1 | model | StrOutputParser()

chain2 = {"city": chain1, "language": itemgetter("language")} | prompt2 | model | StrOutputParser()

chain2.invoke({"person": "obama", "language": "spanish"})

'El país en el que nació la ciudad de Honolulu, Hawái, donde nació Barack Obama, el 44º presidente de los Estados Unidos, es Estados Unidos.'

In [32]:
from langchain.schema.runnable import RunnableMap
prompt1 = ChatPromptTemplate.from_template("generate a random color")
prompt2 = ChatPromptTemplate.from_template("what is a fruit of color: {color}")
prompt3 = ChatPromptTemplate.from_template("what is countries flag that has the color: {color}")
prompt4 = ChatPromptTemplate.from_template("What is the color of {fruit} and {country}")
chain1 = prompt1 | model | StrOutputParser()
chain2 = RunnableMap(steps={"color": chain1}) | {
    "fruit": prompt2 | model | StrOutputParser(),
    "country": prompt3 | model | StrOutputParser(),
} | prompt4

In [33]:
chain2.invoke({})

ChatPromptValue(messages=[HumanMessage(content="What is the color of A fruit that has a color similar to #7E7DE6 is the Peruvian Apple Cactus (Cereus repandus). It is a tropical fruit with a vibrant purple or violet exterior. and The country's flag that has the color #7E7DE6 is North Macedonia.", additional_kwargs={}, example=False)])

## Arbitrary Functions

You can use arbitrary functions in the pipeline

Note that all inputs to these functions need to be a SINGLE argument. If you have a function that accepts multiple arguments, you should write a wrapper that accepts a single input and unpacks it into multiple argument.

In [35]:
from langchain.schema.runnable import RunnableLambda

def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

chain1 = prompt | model

chain = {
    "a": itemgetter("foo") | RunnableLambda(length_function),
    "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")} | RunnableLambda(multiple_length_function)
} | prompt | model

In [36]:
chain.invoke({"foo": "bar", "bar": "gah"})

AIMessage(content='3 + 9 is equal to 12.', additional_kwargs={}, example=False)

## SQL Database

We can also try to replicate our SQLDatabaseChain using this style.

In [37]:
template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}"""
prompt = ChatPromptTemplate.from_template(template)

In [38]:
from langchain.utilities import SQLDatabase

In [41]:
db = SQLDatabase.from_uri("sqlite:///../../../../notebooks/Chinook.db")

In [42]:
def get_schema(_):
    return db.get_table_info()

In [43]:
def run_query(query):
    return db.run(query)

In [47]:
inputs = {
    "schema": RunnableLambda(get_schema),
    "question": itemgetter("question")
}
sql_response = (
        RunnableMap(inputs)
        | prompt
        | model.bind(stop=["\nSQLResult:"])
        | StrOutputParser()
    )

In [48]:
sql_response.invoke({"question": "How many employees are there?"})

'SELECT COUNT(*) \nFROM Employee;'

In [49]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

In [52]:
full_chain = (
    RunnableMap({
        "question": itemgetter("question"),
        "query": sql_response,
    }) 
    | {
        "schema": RunnableLambda(get_schema),
        "question": itemgetter("question"),
        "query": itemgetter("query"),
        "response": lambda x: db.run(x["query"])    
    } 
    | prompt_response 
    | model
)

In [53]:
full_chain.invoke({"question": "How many employees are there?"})

AIMessage(content='There are 8 employees.', additional_kwargs={}, example=False)

## Code Writing

In [57]:
from langchain.utilities import PythonREPL
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [58]:
template = """Write some python code to solve the user's problem. 

Return only python code in Markdown format, eg:

```python
....
```"""
prompt = ChatPromptTemplate(messages=[
    SystemMessagePromptTemplate.from_template(template),
    HumanMessagePromptTemplate.from_template("{input}")
])

In [64]:
def _sanitize_output(text: str):
    _, after = text.split("```python")
    return after.split("```")[0]

In [67]:
chain = prompt | model | StrOutputParser() | _sanitize_output | PythonREPL().run

In [68]:
chain.invoke({"input": "whats 2 plus 2"})

Python REPL can execute arbitrary code. Use with caution.


'4\n'